In [34]:
import numpy as np
import pandas as pd
import nltk
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
#!pip install emoji
import emoji as emoji
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import nltk
import tensorflow_hub as hub
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [28]:
train_X = pd.read_csv('/emojify_train_x.csv',header=None)
test_X = pd.read_csv('/emojiy_test_x.csv',header=None)

train_Y = pd.read_csv('/Emojify_Y_train.csv',header=None)
test_Y = pd.read_csv('/emojiy_y_test.csv',header=None)

In [29]:
frames = [train_X , test_X]
X = pd.concat(frames)
X.shape

(188, 1)

In [30]:
frames = [train_Y , test_Y]
Y = pd.concat(frames)
Y.shape

(188, 1)

In [35]:
import re as s
def clean(train):
  List = train.iloc[:,0].tolist()
  Regex = str.maketrans("","","'")
  word = [s.translate(Regex) for s in List]
  tokenized_sent = []
  for s in word:
    tokenized_sent.append(word_tokenize(s.lower())) 
  return word , tokenized_sent


trained_list , trained_and_tokened_list = clean(X)

In [36]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model_e = hub.load(module_url)
print ("module %s loaded" % module_url)

sentence_embeddings = model_e(trained_list)


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [37]:
category = Y.iloc[:,0].tolist()
category_ohe = pd.get_dummies(category)

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentence_embeddings.numpy(), category_ohe, test_size=0.33, random_state=42)

In [39]:
from keras.layers import *
from keras.models import Sequential
model = Sequential()
model.add(Dense(64, input_dim=512, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(5, activation='sigmoid'))
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['acc'])
model.fit(X_train,y_train,epochs=40,shuffle=True,validation_split=0.1)

Epoch 1/40
4/4 [==============================] - 1s 85ms/step - loss: 1.6031 - acc: 0.2321 - val_loss: 1.5887 - val_acc: 0.3846
Epoch 2/40
4/4 [==============================] - 0s 15ms/step - loss: 1.5606 - acc: 0.4911 - val_loss: 1.5484 - val_acc: 0.3846
Epoch 3/40
4/4 [==============================] - 0s 16ms/step - loss: 1.5212 - acc: 0.5446 - val_loss: 1.5000 - val_acc: 0.4615
Epoch 4/40
4/4 [==============================] - 0s 15ms/step - loss: 1.4667 - acc: 0.5714 - val_loss: 1.4408 - val_acc: 0.4615
Epoch 5/40
4/4 [==============================] - 0s 13ms/step - loss: 1.4002 - acc: 0.5982 - val_loss: 1.3610 - val_acc: 0.6154
Epoch 6/40
4/4 [==============================] - 0s 15ms/step - loss: 1.3150 - acc: 0.6875 - val_loss: 1.2610 - val_acc: 0.7692
Epoch 7/40
4/4 [==============================] - 0s 12ms/step - loss: 1.2130 - acc: 0.7411 - val_loss: 1.1423 - val_acc: 0.7692
Epoch 8/40
4/4 [==============================] - 0s 13ms/step - loss: 1.0949 - acc: 0.8125 - val

In [40]:
model.evaluate(X_test,y_test)

2/2 [==============================] - 0s 6ms/step - loss: 0.4370 - acc: 0.8413


[0.43697020411491394, 0.841269850730896]

In [41]:
import emoji as emoji
#emoji.EMOJI_ALIAS_UNICODE

emoji_dictionary = {"0": "\u2764\uFE0F",    # :heart: prints a black instead of red heart depending on the font
                    "1": ":baseball:",
                    "2": ":beaming_face_with_smiling_eyes:",
                    "3": ":downcast_face_with_sweat:",
                    "4": ":fork_and_knife:",
                   }


In [42]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😁
😓
🍴


In [43]:
query = ['I am gping to temple']
inp = model_e(query)
out =  model.predict(inp).flatten().tolist()

max_value = max(out)
max_index = out.index(max_value)

print(out)
print(max_value)
print(max_index)



print('Statement is :  '+query[0])
print('Classification Result is :  '+emoji.emojize(emoji_dictionary[str(max_index)]))



[0.37604260444641113, 0.6350694894790649, 0.07485029101371765, 0.5461792349815369, 0.5365097522735596]
0.6350694894790649
1
Statement is :  I am gping to temple
Classification Result is :  ⚾
